## Train linear model
This notebook fits linear encoding models to phoneme level data using different stimulus matrices, depending on the analysis in question. Here is a useful shorthand for the different model constructions:
* Model 1: 14 phonological features + 4 task features + normalized EMG
* Model 2: 42 phonological features (14 perception only, 14 production only, 14 combined) + 4 task features + normalized EMG
* Model 3: 14 phonological features + 2 task features (predictability contrast omitted) + normalized EMG
* Model 4: 14 phonological features + 2 task features (perception/production contrast omitted) + normalized EMG

Any model ending in `e` is the same as the above, but _without_ an EMG regressor.

**Warning: this notebook is very computationally intensive.** You may wish to run it on your lab server instead of a local machine. Another alternative is to split the subjects up into smaller subsets, then run those subsets instead of holding all the subjects' models in memory at the same time.

In [1]:
import mne
import re
import numpy as np
from glob import glob
import csv
from matplotlib import cm,rcParams
from matplotlib import pyplot as plt
import pickle
import h5py
from tqdm.notebook import tqdm
import sys
sys.path.append('./utils/')
import textgrid
import strf

In [2]:
# Change these path for running the notebook locally
eeg_data_path = '/path/to/dataset/' # downloadable from OSF: https://doi.org/10.17605/OSF.IO/FNRD9
git_path  = '/path/to/git/speaker_induced_suppression_EEG/'
pkl_path = '/path/to/pickles/' # Where the split data will be saved
h5_path = '/path/to/h5/' # Where the model results will be saved

In [3]:
exclude = ['OP0001','OP0002','OP0020']
subjs = np.sort([s[-6:] for s in glob(f'{git_path}eventfiles/*') if 'OP0' in s and s[-6:] not in exclude])
condition, level = 'all', 'phoneme'
model_number = 'model1'
tmin,tmax = -0.3,0.5
delays = np.arange(np.floor(tmin*128),np.ceil(tmax*128),dtype=int)
nboots = 10
alphas=np.hstack((0, np.logspace(-4,4,20)))
features_dict = {
                'dorsal': ['y','w','k','kcl', 'g','gcl','eng','ng'],
                'coronal': ['ch','jh','sh','zh','s','z','t','tcl','d','dcl','n','th','dh','l','r'],
                'labial': ['f','v','p','pcl','b','bcl','m','em','w'],
                'high': ['uh','ux','uw','iy','ih','ix','ey','eh','oy'],
                'front': ['iy','ih','ix','ey','eh','ae','ay'],
                'low': ['aa','ao','ah','ax','ae','aw','ay','axr','ow','oy'],
                'back': ['aa','ao','ow','ah','ax','ax-h','uh','ux','uw','axr','aw'],
                'plosive': ['p','pcl','t','tcl','k','kcl','b','bcl','d','dcl','g','gcl','q'],
                'fricative': ['f','v','th','dh','s','sh','z','zh','hh','hv','ch','jh'],
                'syllabic': ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux'],
                'nasal': ['m','em','n','en','ng','eng','nx'],
                'voiced':   ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux','w','y','el','l','r','dh','z','v','b','bcl','d','dcl','g','gcl','m','em','n','en','eng','ng','nx','q','jh','zh'],
                'obstruent': ['b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx','f', 'g', 'gcl', 'hh', 'hv','jh', 'k', 'kcl', 'p', 'pcl', 'q', 's', 'sh','t', 'tcl', 'th','v','z', 'zh','q'],
                'sonorant': ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux','w','y','el','l','r','m', 'n', 'ng', 'eng', 'nx','en','em'],
        }
features = [f for f in features_dict.keys()]
print(features)
print(len(features))

['dorsal', 'coronal', 'labial', 'high', 'front', 'low', 'back', 'plosive', 'fricative', 'syllabic', 'nasal', 'voiced', 'obstruent', 'sonorant']
14


In [4]:
# load raw and emg info from ica.fif (assuming we want it in our model)
raws, emgs = dict(), dict()
for s in tqdm(subjs):
    blockid = f'{s}_B1'
    fname = f'{eeg_data_path}{s}/{blockid}/{blockid}_cca.vhdr'
    raws[s] = mne.io.read_raw_brainvision(fname,preload=True,verbose=False)
    if model_number[-1] != 'e': # if we want EMG, get it
        ica_fname = f'{eeg_data_path}{s}/{blockid}/{blockid}_ica.fif'
        ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)
        if 'hEOG' in ica_raw.info['ch_names']:
            ica_raw.notch_filter(60,picks=['hEOG'],verbose=False)
            ica_raw.filter(l_freq=1,h_freq=None,picks=['hEOG'],verbose=False)
            emg = ica_raw.get_data(picks=['hEOG'])
            emgs[s] = emg/np.abs(emg).max() # Scale EMG
        else:
            raise Exception('EMG not found')

  0%|          | 0/18 [00:00<?, ?it/s]

Opening raw data file /home/kfsh/data/eeg/OP0003/OP0003_B1/OP0003_B1_ica.fif...
    Range : 0 ... 516273 =      0.000 ...  4033.383 secs
Ready.
Reading 0 ... 516273  =      0.000 ...  4033.383 secs...


/tmp/ipykernel_16062/139556513.py:9: RuntimeWarning: This filename (/home/kfsh/data/eeg/OP0003/OP0003_B1/OP0003_B1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)


Opening raw data file /home/kfsh/data/eeg/OP0004/OP0004_B1/OP0004_B1_ica.fif...
    Range : 0 ... 639525 =      0.000 ...  4996.289 secs
Ready.
Reading 0 ... 639525  =      0.000 ...  4996.289 secs...


/tmp/ipykernel_16062/139556513.py:9: RuntimeWarning: This filename (/home/kfsh/data/eeg/OP0004/OP0004_B1/OP0004_B1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)


...

/tmp/ipykernel_16062/139556513.py:9: RuntimeWarning: This filename (/home/kfsh/data/eeg/OP0018/OP0018_B1/OP0018_B1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)


Opening raw data file /home/kfsh/data/eeg/OP0019/OP0019_B1/OP0019_B1_ica.fif...
    Range : 0 ... 393951 =      0.000 ...  3077.742 secs
Ready.
Reading 0 ... 393951  =      0.000 ...  3077.742 secs...


/tmp/ipykernel_16062/139556513.py:9: RuntimeWarning: This filename (/home/kfsh/data/eeg/OP0019/OP0019_B1/OP0019_B1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)


Opening raw data file /home/kfsh/data/eeg/OP0021/OP0021_B1/OP0021_B1_ica.fif...
    Range : 0 ... 453451 =      0.000 ...  3542.586 secs
Ready.
Reading 0 ... 453451  =      0.000 ...  3542.586 secs...


/tmp/ipykernel_16062/139556513.py:9: RuntimeWarning: This filename (/home/kfsh/data/eeg/OP0021/OP0021_B1/OP0021_B1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_raw = mne.io.read_raw_fif(ica_fname, preload=True)


In [5]:
# get events
samps = dict()
for s in tqdm(subjs):
    blockid = f'{s}_B1'
    samps[s] = dict()
    for ch in ['spkr','mic']:
        samps[s][ch] = dict()
        for l in ['sn','wr','ph']:
            samps[s][ch][l] = dict()
            for co in ['el','sh','all']:
                if s == 'OP0015' and co == 'sh':
                    print(f'{s} has no {co} events, skipping...')
                else:
                    event_fpath = f'{git_path}eventfiles/{s}/{blockid}/{blockid}_{ch}_{l}_{co}.txt'
                    event_file = []
                    with open(event_fpath,'r') as csvfile:
                        csvReader = csv.reader(csvfile,delimiter='\t')
                        for row in csvReader:
                            if l == 'ph':
                                event_file.append(row[:3]+row[4:])
                            else:
                                event_file.append(row[:3])
                    event_file = np.array(event_file,dtype=float)
                    event_samp = event_file.copy()
                    sfreq = raws[s].info['sfreq']
                    event_samp[:,:2] = np.round(event_file[:,:2]*sfreq)
                    # We need to remove non task samples from mic
                    if ch == 'mic':
                        tg_path = f'{git_path}textgrids/{s}/{blockid}/{blockid}_mic.textgrid'
                        with open(tg_path) as r:
                            tg = textgrid.TextGrid(r.read())
                        task_times = []
                        for row in tg.tiers[2].simple_transcript:
                            if row[2] == 'task':
                                task_times.append([float(row[0])*sfreq,float(row[1])*sfreq])
                        task_times = np.array(task_times,dtype=int)                            
                        task_range = [np.arange(t[0],t[1],step=1) for t in task_times]
                        trial_samps = []
                        for rng in task_range:
                            for samp in event_samp.astype(int):
                                if samp[0] in rng and samp[1] in rng:
                                    trial_samps.append(samp)
                        event_samp = np.array(trial_samps)
                    samps[s][ch][l][co] = event_samp.astype(int)                  

  0%|          | 0/18 [00:00<?, ?it/s]

OP0015 has no sh events, skipping...
OP0015 has no sh events, skipping...
OP0015 has no sh events, skipping...
OP0015 has no sh events, skipping...
OP0015 has no sh events, skipping...
OP0015 has no sh events, skipping...


In [6]:
# Now that we have our samples, get our stim and resp
phonemes = np.loadtxt('./phonemes.txt',dtype=str)
stims,resps = dict(),dict()
for s in tqdm(subjs):
    blockid = f'{s}_B1'
    stims[s],resps[s] = dict(),dict()
    picks = mne.pick_types(raws[s].info, meg=False,eeg=True,stim=False)
    resps[s] = raws[s].get_data(picks=picks).T
    phn_stim_spkr = np.zeros((resps[s].shape[0],len(phonemes)))
    feat_stim_spkr = np.zeros((resps[s].shape[0],len(features)))
    phn_stim_mic = np.zeros((resps[s].shape[0],len(phonemes)))
    feat_stim_mic = np.zeros((resps[s].shape[0],len(features)))
    el_sh_stim = np.zeros((resps[s].shape[0],2))
    # Get timing of predictable/unpredictable segments
    ev_fpath = f'{git_path}eventfiles/{s}/{blockid}/{blockid}_'
    ev_files = [ev_fpath+'spkr_sn_el.txt',ev_fpath+'mic_sn_el.txt',
                ev_fpath+'spkr_sn_sh.txt',ev_fpath+'mic_sn_sh.txt']
    el_times,sh_times = [],[]
    for f in ev_files:
        with open(f,'r') as c:
            cr = csv.reader(c,delimiter='\t')
            for row in cr:
                onset = row[0]
                offset = row[1]
                if 'el' in f:
                    el_times.append([onset,offset])
                elif 'sh' in f:
                    sh_times.append([onset,offset])
    el_times = np.array(np.array(el_times,dtype=float)*128,dtype=int)
    sh_times = np.array(np.array(sh_times,dtype=float)*128,dtype=int)
    for samp in samps[s]['spkr']['ph']['all']:
        onset = samp[0]
        # get el/sh timing
        for el_time in el_times:
            if onset >= el_time[0] and onset <= el_time[1]:
                el_sh_stim[onset,0] = 1
        for sh_time in sh_times:
            if onset >= sh_time[0] and onset <= sh_time[1]:
                el_sh_stim[onset,1] = 1
        # get label
        phn_label = samp[2]
        phn_stim_spkr[onset,phn_label]=1
        phn_stripped = re.sub(r'\d+', '', phonemes[phn_label].lower())
        for fi, f in enumerate(features):
            if phn_stripped in features_dict[f]:
                feat_stim_spkr[onset,fi] = 1
    for samp in samps[s]['mic']['ph']['all']:
        onset = samp[0]
        phn_label = samp[2]
        phn_stim_mic[onset,phn_label]=1
        phn_stripped = re.sub(r'\d+', '', phonemes[phn_label].lower())
        for fi, f in enumerate(features):
            if phn_stripped in features_dict[f]:
                feat_stim_mic[onset,fi] = 1
    any_phn_stim = np.vstack((phn_stim_spkr, phn_stim_mic)).sum(0)
    any_phn_stim = (np.atleast_2d(any_phn_stim)).T
    if 'model1' in model_number:
        stims[s] = np.hstack(((feat_stim_spkr+feat_stim_mic),
                                np.atleast_2d(phn_stim_spkr.sum(1)).T,
                                np.atleast_2d(phn_stim_mic.sum(1)).T,
                                el_sh_stim)).T
    if 'model2' in model_number:
        print(s, "creating model two")
        stims[s] = np.hstack(((feat_stim_spkr + feat_stim_mic), 
                              feat_stim_spkr, feat_stim_mic,
                              np.atleast_2d(phn_stim_spkr.sum(1)).T, 
                              np.atleast_2d(phn_stim_mic.sum(1)).T,el_sh_stim)).T
    if 'model3' in model_number:
        print(s, "creating model three")
        stims[s] = np.hstack(((feat_stim_spkr + feat_stim_mic), 
                              np.atleast_2d(phn_stim_spkr.sum(1)).T, 
                              np.atleast_2d(phn_stim_mic.sum(1)).T)).T
    if 'model4' in model_number:
        print(s, "creating model four")
        stims[s] = np.hstack(((feat_stim_spkr + feat_stim_mic),el_sh_stim)).T
    # Add the emg
    if len(emgs.keys()) > 0:      
        try:
            stims[s] = np.vstack((stims[s],emgs[s]))
        except:
            warnings.warn(f'EMG for {s} not applied to model! Again, something is wrong...')
print(stims[s].shape) # feats x samps
print(resps[s].shape) # samps x chs

  0%|          | 0/18 [00:00<?, ?it/s]

(19, 453452)
(453452, 64)


In [7]:
# split stim and resp into training and validation sets
# we need to split them according to sentence ID to avoid double-dipping
tStims,tResps,vStims,vResps = dict(),dict(),dict(),dict()
for s in tqdm(subjs):
    blockid = f'{s}_B1'
    fpath = pkl_path + f'{model_number}/{blockid}_'
    if len(glob(fpath + '*Stim*')) > 0:
        print(f"Data already split for {blockid}, loading now...")
        with open(fpath+'tResp.pkl','rb') as f:
            tResps[s] = pickle.load(f)
        with open(fpath+'vResp.pkl','rb') as f:
            vResps[s] = pickle.load(f)
        with open(fpath+'tStim.pkl','rb') as f:
            tStims[s] = pickle.load(f)
        with open(fpath+'vStim.pkl','rb') as f:
            vStims[s] = pickle.load(f)
    else:
        sen_onset,sen_offset,sen_id = [],[],[]
        for ch in ['spkr','mic']:
            ev_fname = f'{git_path}eventfiles/{s}/{blockid}/{blockid}_{ch}_sn_all.txt'
            with open(ev_fname,'r') as my_csv:
                csvReader = csv.reader(my_csv,delimiter='\t')
                for row in csvReader:
                    sen_onset.append(float(row[0]))
                    sen_offset.append(float(row[1]))
                    sen_id.append(int(row[2]))
        sen_samps = dict() # sample ranges for each unique ID
        for unique_idx, unique_id in enumerate(np.unique(sen_id)):
            sen_range = []
            for this_idx, this_id in enumerate(sen_id):
                if this_id == unique_id: # if we have a match
                    onset_samp = int(sen_onset[this_idx]*sfreq)
                    offset_samp = int(sen_offset[this_idx]*sfreq)
                    sen_range.append([onset_samp,offset_samp])
            sen_samps[unique_id] = sen_range
        jitter = 0.5
        resp_dict,stim_dict = dict(),dict()
        for unique_id in sen_samps.keys():
            resp_matches, stim_matches = [], []
            for unique_idx, times in enumerate(sen_samps[unique_id]):
                onset_samp, offset_samp = times[0], times[1]
                for samp_idx in range(len(resps[s])):
                    if samp_idx >= onset_samp-jitter and samp_idx <= offset_samp+jitter:
                        resp_matches.append(resps[s][samp_idx,:])
                        stim_matches.append(stims[s][:,samp_idx])
            resp_dict[unique_id] = np.array(resp_matches)
            stim_dict[unique_id] = np.array(stim_matches)
        tval_cutoff = 40 # 40/50 = 80% train
        np.random.seed(6655321)
        train_sen_ids = np.random.permutation(list(resp_dict.keys()))[:tval_cutoff]
        np.random.seed(6655321)
        val_sen_ids = np.random.permutation(list(resp_dict.keys()))[tval_cutoff:]
        tStims_by_sen,vStims_by_sen,tResps_by_sen,vResps_by_sen = dict(),dict(),dict(),dict()
        for train_id in train_sen_ids:
            tResps_by_sen[train_id] = resp_dict[train_id]
            tStims_by_sen[train_id] = stim_dict[train_id]
        for val_id in val_sen_ids:
            vResps_by_sen[val_id] = resp_dict[val_id]
            vStims_by_sen[val_id] = stim_dict[val_id]
        tStims[s] = np.vstack(list(tStims_by_sen.values()))
        vStims[s] = np.vstack(list(vStims_by_sen.values()))
        tResps[s] = np.vstack(list(tResps_by_sen.values()))
        vResps[s] = np.vstack(list(vResps_by_sen.values()))
        print(f"{blockid}: training on {tStims[s].shape[0]} samples, validating on {vStims[s].shape[0]} samples. raw contained {stims[s].shape[1]} samples,")
        if tStims[s].shape[0] != tResps[s].shape[0]:
                print(tStims[s].shape, tResps[s].shape)
                raise Exception("Stim and resp do not have the same shape! (training)")
        if vStims[s].shape[0] != vResps[s].shape[0]:
                print(vStims[s].shape, vResps[s].shape)
                raise Exception("Stim and resp do not have the same shape! (validation)")
        # Save to h5
        with open(fpath+'tResp.pkl','wb') as f:
            pickle.dump(tResps[s],f)
        with open(fpath+'vResp.pkl','wb') as f:
            pickle.dump(vResps[s],f)
        with open(fpath+'tStim.pkl','wb') as f:
            pickle.dump(tStims[s],f)
        with open(fpath+'vStim.pkl','wb') as f:
            pickle.dump(vStims[s],f)

  0%|          | 0/18 [00:00<?, ?it/s]

OP0003_B1: training on 164714 samples, validating on 39396 samples. raw contained 516274 samples,
OP0004_B1: training on 206994 samples, validating on 51644 samples. raw contained 639526 samples,
OP0005_B1: training on 154436 samples, validating on 39292 samples. raw contained 409081 samples,
OP0006_B1: training on 189840 samples, validating on 48277 samples. raw contained 545632 samples,
OP0007_B1: training on 176129 samples, validating on 45315 samples. raw contained 478278 samples,
OP0008_B1: training on 127526 samples, validating on 32749 samples. raw contained 412851 samples,
OP0009_B1: training on 149277 samples, validating on 38696 samples. raw contained 407744 samples,
OP0010_B1: training on 143997 samples, validating on 39513 samples. raw contained 429331 samples,
OP0011_B1: training on 137337 samples, validating on 34678 samples. raw contained 378214 samples,
OP0012_B1: training on 155049 samples, validating on 38165 samples. raw contained 397382 samples,
OP0013_B1: training 

In [9]:
# This cell takes an incredibly long time to run
for s in tqdm(subjs):
    blockid = s + '_B1'
    # Fit the model
    corrs,wts,dtStim,dtResp,dvStim,dvResp,valphas = strf.strf(
        tResps[s],tStims[s],nboots=nboots, delay_min=tmin, delay_max=tmax,
        vResp=vResps[s], vStim=vStims[s],flip_resp=True,alphas=alphas)
    # Save model results to h5 file
    with h5py.File(f'{h5_path}{model_number}/{s}_model.hdf5','w') as f:
        f.create_dataset('wts',data=wts[s][0].reshape((tStims[s].shape[1],-1,dvResp[s].shape[1])))
        f.create_dataset('corrs',data=corrs[s][0])
        f.create_dataset('alphas',data=valphas[s])
        f.create_dataset('dtStim',data=dtStim[s])
        f.create_dataset('dtResp',data=dtResp[s])
        f.create_dataset('dvStim',data=dvStim[s])
        f.create_dataset('dvResp',data=dvResp[s])

  0%|          | 0/18 [00:00<?, ?it/s]

62 164714
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(105815, 1957)
Covmat shape: 
(1957, 1957)
(1957, 1957) (1957, 105815) (105815, 62)


INFO:counter:1/10 items complete (8.65 seconds/item, 00:01:17 remaining)


Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(105815, 1957)
Covmat shape: 
(1957, 1957)
(1957, 1957) (1957, 105815) (105815, 62)


INFO:counter:2/10 items complete (8.80 seconds/item, 00:01:10 remaining)


...

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(119063, 1957)
Covmat shape: 
(1957, 1957)
(1957, 1957) (1957, 119063) (119063, 64)


INFO:counter:9/10 items complete (10.57 seconds/item, 00:00:10 remaining)


Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(119063, 1957)
Covmat shape: 
(1957, 1957)
(1957, 1957) (1957, 119063) (119063, 64)


INFO:counter:10/10 items complete (10.54 seconds/item, 00:00:00 remaining)


Cmode = False
Number of time points is greater than the number of features
stim shape (not cmode): 
(148315, 1957)
Covmat shape: 
(1957, 1957)
(1957, 1957) (1957, 148315) (148315, 64)
wt shape:
(1957, 64)
(1957, 64)
